In [2]:
import pandas as pd
import numpy as np

In [3]:
db_ranking = pd.read_csv('dbRanking.csv', delimiter='|')
db_ranking.sample(5)

,taxpayer_id,company_economic_activity_id,ciiu_spanish_desc,spanish_sector_company_name,employees_number,branches_number,activity_start_date,seniority_company_years_number,company_size_desc,foreign_company_mark_type,...,pbi_sector_2015_per,pbi_sector_2014_per,pbi_sector_2013_per,pbi_sector_2012_per,pbi_sector_2011_per,pbi_sector_2010_per,pbi_sector_2009_per,pbi_sector_2008_per,pbi_sector_2007_per,cutoff_date
6926,39103473652,7491,ObtenciÃ³n y dotaciÃ³n de personal,CONSULTORIAS Y SERVICIOS,NaN,1.0,1993-08-12,25.0,PequeÃ±a empresa / Small-size Company,NaN,...,1.8,5.5,4.9,5.90,6.71,8.82,9.69,-0.4,12.99,2020-02-28
9853,88246002168,5150,"Venta la por mayor de maquinaria, equipo y mat...",COMERCIALIZADORAS,14.0,1.0,1996-11-01,22.0,PequeÃ±a empresa / Small-size Company,NaN,...,3.4,2.3,0.0,5.63,6.80,6.90,10.70,-0.7,8.90,2020-02-28
6120,1486145710,8030,EnseÃ±anza superior,EDUCACION,9.0,1.0,1995-09-01,23.0,PequeÃ±a empresa / Small-size Company,NaN,...,2.9,4.5,3.1,3.60,4.90,3.50,1.70,0.7,2.20,2020-02-28
2558,35654760803,500,"Pesca, explotaciÃ³n de criaderos de peces y gr...",PESCA,71.0,1.0,1984-03-14,34.0,Mediana empresa / Medium-size Company,NaN,...,1.8,3.9,4.4,5.80,6.70,8.80,9.69,-0.9,12.80,2020-02-28
2860,36100359752,4520,ConstrucciÃ³n de edificios completos o de part...,CONSTRUCTORAS,NaN,1.0,2013-05-02,1.0,PequeÃ±a empresa / Small-size Company,NaN,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,2020-02-28


In [4]:
ranking_dict = ['ranking_'+str(i)+'_number' for i in range(2017,2009,-1)]
min_usd_billing_dict = ['min_usd_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
max_usd_billing_dict = ['max_usd_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
min_pen_billing_dict = ['min_pen_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)] 
max_pen_billing_dict = ['max_pen_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
pbi_sector_dict = ['pbi_sector_'+str(i)+'_per' for i in range(2016,2006,-1)]

def clean_db_ranking(df):
    df = df.drop_duplicates(ignore_index=True)
    df = df.dropna(axis=1, how='all')
    df = df[df['market_share_per'] < 99] # Removing, these are like 10 registers
    df = df.drop(['ebitda_2014_amount', 'ebitda_2013_amount'], axis=1) # Removing these columns because are too few
    
    for col in ranking_dict: #computing and droping ranking
        df['ranking_2017_number'].fillna(df[col], inplace=True)
    df['ranking_last_number'] = df['ranking_2017_number']
    df = df.drop(ranking_dict, axis=1)
    
    for col in min_usd_billing_dict: #computing and dropping min usd amounts
        df['min_usd_billing_2017_amount'].fillna(df[col], inplace=True)
    df['min_usd_billing_last_amount'] = df['min_usd_billing_2016_amount']
    df = df.drop(min_usd_billing_dict, axis=1)

    for col in max_usd_billing_dict: #computing and dropping max usd amounts
        df['max_usd_billing_2017_amount'].fillna(df[col], inplace=True)
    df['max_usd_billing_last_amount'] = df['max_usd_billing_2017_amount']
    df = df.drop(max_usd_billing_dict, axis=1)

    for col in min_pen_billing_dict: #computing and dropping max usd amounts
        df['min_pen_billing_2017_amount'].fillna(df[col], inplace=True)
    df['min_pen_billing_last_amount'] = df['min_pen_billing_2017_amount']
    df = df.drop(min_pen_billing_dict, axis=1)

    for col in max_pen_billing_dict: #computing and dropping max usd amounts
        df['max_pen_billing_2017_amount'].fillna(df[col], inplace=True)
    df['max_pen_billing_last_amount'] = df['max_pen_billing_2017_amount']
    df = df.drop(max_pen_billing_dict, axis=1)

    df['exporter_mark_type_flag'] = df['exporter_mark_type']
    df['importer_mark_type_flag'] = df['importer_mark_type']

    df = df.drop(df.columns[[i for i in range(15, 185)]], axis=1)

    for col in pbi_sector_dict: #computing and dropping max usd amounts
        df['pbi_sector_2016_per'].fillna(df[col], inplace=True)
    df['pbi_sector_last_per'] = df['pbi_sector_2016_per']
    df['pbi_sector_last_per'] = df['pbi_sector_last_per'].replace(0, np.nan)
    df = df.drop(pbi_sector_dict, axis=1)

    df = df.drop(['cutoff_date'], axis=1)

    return df

In [5]:
db_ranking = clean_db_ranking(db_ranking)
db_ranking.sample(5)

,taxpayer_id,company_economic_activity_id,ciiu_spanish_desc,spanish_sector_company_name,employees_number,branches_number,activity_start_date,seniority_company_years_number,company_size_desc,foreign_company_mark_type,...,private_public_mark_desc,market_share_per,ranking_last_number,min_usd_billing_last_amount,max_usd_billing_last_amount,min_pen_billing_last_amount,max_pen_billing_last_amount,exporter_mark_type_flag,importer_mark_type_flag,pbi_sector_last_per
98,10445448714,6021,Otros tipos de transporte regular de pasajeros...,TURISMO,34.0,1.0,2005-02-16,13.0,Mediana empresa / Medium-size Company,NaN,...,PRIVADA,0.234204,8536.0,3659259.0,3218883.814,9900000.0,10499999.0,NaN,IMPORTADORA,2.4
5223,274002229,8511,Actividades de hospitales,SALUD,NaN,1.0,9999-12-31,1.0,PequeÃ±a empresa / Small-size Company,NaN,...,PRIVADA,0.248629,7646.0,NaN,3884572.000,9500000.0,10499999.0,NaN,NaN,2.4
5934,20408838433,7421,Actividades de arquitectura e ingenierÃ­a y ac...,CONSULTORIAS Y SERVICIOS,82.0,1.0,2011-07-01,7.0,Mediana empresa / Medium-size Company,NaN,...,PRIVADA,0.059811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.8
388,31341336964,2221,Actividades de impresiÃ³n,IMPRESIÃ“N,13.0,1.0,1999-05-10,19.0,Mediana empresa / Medium-size Company,NaN,...,PRIVADA,0.433709,9779.0,NaN,2856246.000,8700000.0,9099999.0,NaN,IMPORTADORA,1.9
3838,3178406028,1512,ElaboraciÃ³n y conservaciÃ³n de pescado y prod...,PESCA,32.0,1.0,2009-08-17,9.0,PequeÃ±a empresa / Small-size Company,NaN,...,PRIVADA,0.172852,3805.0,NaN,8879023.000,20000000.0,23999999.0,NaN,IMPORTADORA,3.0


In [6]:
db_ranking['spanish_sector_company_name'].value_counts().index.tolist()

['COMERCIALIZADORAS',
 'CONSULTORIAS Y SERVICIOS',
 'CONSTRUCTORAS',
 'TRANSPORTE',
 'METALMECANICA',
 'AGROINDUSTRIA Y GANADERIA',
 'INMOBILIARIAS',
 'TURISMO',
 'OTROS SERVICIOS',
 'HIDROCARBUROS',
 'INFORMATICA',
 'VEHICULOS',
 'TEXTILES',
 'MINERIA',
 'INDUSTRIA ALIMENTARIA',
 'EDUCACION',
 'CAUCHO Y PLASTICO',
 'TELECOMUNICACIONES',
 'PESCA',
 'SALUD',
 'ALQUILERES (MAQUINARIA)',
 'INTERMEDIACION FINANCIERA',
 'ESPARCIMIENTO',
 'AUTOMOTRIZ',
 'ABOGADOS',
 'QUIMICAS',
 'IMPRESIÃ“N',
 'MINERALES NO METALICOS',
 'FORESTAL Y SILVICULTURA',
 'ORGANIZACIONES',
 'ENERGIA Y AGUA',
 'EDITORIALES',
 'ADMINISTRACION PUBLICA',
 'PAPEL',
 'CURTIDURIA',
 'CORREOS',
 'RECICLAJE',
 'LABORATORIOS',
 'BEBIDAS',
 'INVESTIGACION Y DESARROLLO',
 'VETERINARIAS']

In [9]:
db_ranking['ciiu_spanish_desc'].value_counts()

ConstrucciÃ³n de edificios completos o de partes de edificios, obras de ingenierÃ­a civil       538
Venta al por mayor de otros productos                                                           522
Otras actividades empresariales n.c.p.                                                          349
Transporte de carga por carretera                                                               294
Actividades de arquitectura e ingenierÃ­a y actividaes conexas de asesoramiento tÃ©cnico        222
                                                                                               ... 
ReproducciÃ³n de grabaciones                                                                      1
Actividades de fotografÃ­a                                                                        1
FabricaciÃ³n de cojines, engranajes, trenes de engranajes y piezas de transmisiÃ³n                1
FabricaciÃ³n de artÃ­culos de cuchillerÃ­a, herramientas de mano y artÃ­culos de ferreterÃ­a      1


In [10]:
db_ranking['company_size_desc'].value_counts()

PequeÃ±a empresa / Small-size Company    3093
Mediana empresa / Medium-size Company    2648
Gran Empresa / Big-size Company            28
Name: company_size_desc, dtype: int64

In [17]:
db_ranking['department_name'].value_counts()

Lima                4167
La Libertad          190
Arequipa             176
Piura                131
Ancash               126
Ucayali              121
Lambayeque            98
JunÃ­n                83
Cajamarca             81
Loreto                77
Cusco                 75
San MartÃ­n           68
Puno                  64
HuÃ¡nuco              64
Ica                   62
Ayacucho              40
Tacna                 33
Madre de Dios         25
Tumbes                23
Pasco                 23
Amazonas              22
Moquegua              11
ApurÃ­mac              9
Huancavelica           3
Callao                 2
Av. JosÃ© GÃ¡lve       2
Madre De Dios          1
Name: department_name, dtype: int64